In [1]:
import pandas as pd
import requests
import psycopg2
from psycopg2 import sql


### Extract customer id and email from the table customer

In [2]:
db_params = {
            "host":'localhost',
            "database":'postgres',
            "user":'postgres',
            "password":'!q2W3e4r'
            }

connection = psycopg2.connect(**db_params)

# the actual name of your PostgreSQL table
table_name = 'customers'
column_name1 = 'id'
colum_name2 = 'contact_email'

# Construct the SQL query to select all columns from the table
query = f"SELECT {column_name1},{colum_name2} FROM {table_name}"

# Use pandas to read the data into a DataFrame
df = pd.read_sql_query(query, connection)

# Display the DataFrame
print(df)

# Close the database connection
connection.close()

df_backup = df.copy()


           id                      contact_email
0     1055178                    udrub@krebs.com
1     1037261                   juri80@roemer.de
2     1047267                 iwan95@soelzer.com
3     1050123                 gflantz@stolze.com
4     1023075                   nkabus@beier.com
...       ...                                ...
3696  1021042     nicole66@caldwell-crawford.biz
3697  1057037    christopher90@murphy-morrow.com
3698  1090115  racheldominguez@stevens-evans.net
3699  1025342                 mannlisa@drake.com
3700  1036675              jennifer20@palmer.biz

[3701 rows x 2 columns]


### Extract the domain name and get the Industry values from the API

In [3]:
def extractDomain(x):
    return x.strip().split('@')[1]

email = df['contact_email'].apply(extractDomain)
email.drop_duplicates(inplace=True)
email = email.reset_index(drop=True)

df = pd.DataFrame(email)
df['industry'] = None

In [17]:

def fetchIndustry(df,df_merge,batchno,batchsize):
    url = "https://api.apollo.io/v1/organizations/enrich"
    
    headers = {
        'Cache-Control': 'no-cache',
        'Content-Type': 'application/json'
    }
    df = df.loc[batchsize*batchno:(batchsize*(batchno+1)-1),]
    for i in range(batchsize*batchno,batchsize*(batchno+1)):
        domain = df.loc[i,'contact_email'] 
        querystring = {
            "api_key": "_JPx70RpHyQKBfUB4kAR0Q",
            "domain": domain
        }
        try:
            response = requests.request("GET", url, headers=headers, params=querystring)
            if response.status_code == 200 and bool(response.json()):
                industry = response.json()
                if bool(industry["organization"]["industry"]):
                    print(i)
                    df.loc[i,'industry'] = industry["organization"]["industry"]
            elif response.status_code == 200:
                print(i)
                df.loc[i,'industry'] = None
            elif response.status_code == 429:
                print("API requests exhausted")
            else:
                print(response.text)
        except IOError as e:
        # Handle a more general exception (if any other unexpected exception occurs)
            print(f"Unexpected error: {e}")
        except Exception as e:
        # Print the exact error message
            print(f"An error occurred: {e}")

    df_merge = pd.concat([df_merge, df])
    df_merge.to_csv('CustidandIndustryInfo.csv',header=1,index=0)




In [20]:
# df_merge = pd.DataFrame()
df_merge = pd.read_csv('CustidandIndustryInfo.csv')
batchno = int(len(df_merge)/50)
manualbatchno = 38
if batchno == manualbatchno:
    fetchIndustry(df,df_merge,manualbatchno,50)
else:
    print("check batch no")

#completedbatch 50 : 0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,29,30,31,32,33,34,35,36,37,

1850
1851


c:\Users\abhij\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


1852
1853
1854
1855
1856
1857
1858
1859
1860
1861
1862
1863
1864
1865
1866
1867
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899


In [19]:
# 
df_merge = pd.read_csv('CustidandIndustryInfo.csv')
# df_merge = df_merge.loc[0:249,]
# df_merge = df_merge.where(pd.notnull(df_merge), None)
# df_merge.loc[599,'industry'] = 'financial services'
# df_merge.to_csv('CustidandIndustryInfo.csv',header=1,index=0)
print(df_merge)
# batchno = len(df_merge)/50
# print(int(batchno) )


             contact_email      industry
0                  rust.de   hospitality
1             bolander.com  construction
2               scheibe.de   real estate
3              steckel.com           NaN
4                plath.org           NaN
...                    ...           ...
1845           coppens.org           NaN
1846           demeyer.org           NaN
1847            dubois.com           NaN
1848             smits.biz           NaN
1849  horemans-willems.com           NaN

[1850 rows x 2 columns]


In [247]:
df_backup['domain'] = df_backup['contact_email'].apply(extractDomain)
df_insert = pd.merge(df_backup,df_merge,left_on='domain',right_on='contact_email',how="left")
df_insert = df_insert[['id','industry']]
df_insert.drop_duplicates(inplace=True)
df_insert = df_insert.where(pd.notnull(df_insert), None)
# df_insert.fillna('')

In [248]:
#add new column from pandas df
db_params = {
            "host":'localhost',
            "database":'postgres',
            "user":'postgres',
            "password":'!q2W3e4r'
            }

connection = psycopg2.connect(**db_params)
cursor = connection.cursor()

# Replace 'your_table_name' with the actual name of your PostgreSQL table
table_name = 'customers'
new_column_name = 'industry'
data_type = 'VARCHAR(100)'  # Adjust the data type based on your requirements

# Construct the SQL query to add the new column
query = f"ALTER TABLE {table_name} ADD COLUMN {new_column_name} {data_type};"

# Execute the query
cursor.execute(query)

# Commit the changes
connection.commit()


# Replace 'your_table_name' with the actual name of your PostgreSQL table


# Assuming 'primary_key' is the primary key column in both the DataFrame and the PostgreSQL table
for index, row in df_insert.iterrows():
    primary_key_value = row['id']
    new_column_value = row['industry']

    # Construct the SQL query
    query = f"UPDATE {table_name} SET industry = %s WHERE id = %s"
    values = (new_column_value, primary_key_value)

    # Execute the query
    cursor.execute(query, values)

# Commit the changes and close the connection
connection.commit()
cursor.close()
connection.close()


In [231]:
# df_insert = df_insert.where(pd.isna(df), None)
# print(df_insert)

# df_insert.drop_duplicates(inplace=True)
# print(len(df_insert))
# df_insert[df_insert['id'].isnull()]

# print(df)
# df.to_csv('CustidandIndustry1.csv',header=1,index=0)